In [ ]:
from rag.langchain_vector import markdown_rag
import re

def normalize(name):
    return re.sub(r"[-_. ]+", "-", name).lower()

md_files = "C:\\Users\\shive\\OneDrive\\Documents\\DnD\\Adventures\\Stone-Heart Hollow"
name = normalize(md_files.split("\\")[-1])

print(name)
retriever = markdown_rag(
    source_directory=md_files, collection_name=name
)


In [ ]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from rag.langchain_vector import markdown_rag
import re
import shutil

try:
    db_path = "markdown_db"
    shutil.rmtree(db_path)
except:
    print("no old db")

template = """
You are an assistaint to a Dungeon Master. You will assist them in preparing and running Dungeons and Dragons sessions. 

Here are the relevant documents: {documents}

Here is their query: {question}
"""

model = OllamaLLM(model="gemma3")
prompt = ChatPromptTemplate.from_template(template)

chain = prompt | model

md_files = "C:\\Users\\shive\\OneDrive\\Documents\\DnD\\Adventures\\Stone-Heart Hollow"

def normalize(name):
    return re.sub(r"[-_. ]+", "-", name).lower()

name = normalize(md_files.split("\\")[-1])

retriever = markdown_rag(source_directory=md_files, collection_name=name, db_path=db_path)

while True:
    print("\n-------------------------------")
    question = input("Ask your question (q to quit): ")
    print("\n\n")
    if question == "q":
        break
    
    docs = retriever.invoke(question)
    result = chain.invoke({"documents": docs, "question": question})
    print(f"DMI: {result}")

In [ ]:
from rag.vector import ChromaRag
import os

# --- Testing ---
MODEL_EMBED = "mxbai-embed-large"
MODEL_CHAT = "gemma3"
DB_PATH = "markdown_db"
SOURCE_DIR = r"C:\Users\shive\OneDrive\Documents\DnD\Adventures\Stone-Heart Hollow"
TOP_K = 5


collection_name = os.path.basename(SOURCE_DIR)
chrome_rag = ChromaRag(
    source_directory=SOURCE_DIR,
    collection_name=collection_name,
    db_path=DB_PATH,
    model_name=MODEL_EMBED
)

SYSTEM_PROMPT = (
""""You are an assistant to a Dungeon Master.
    You will assist them in preparing and running Dungeons & Dragons sessions.
    You may also have to answer questions based on what is in the documents.
"""
)
question = "test"
docs = chrome_rag.retrieve(question)

In [ ]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import os
import instructor
from rag.vector import ChromaRag

class Character(BaseModel):
    name: str
    description: int
    fact: List[str] = Field(..., description="A list of facts about the character")

class Answer(BaseModel):
    answer: str = Field(..., description="Exact requirement as per the user's query.")
    references: List[str] = Field(..., description="Key points or facts pulled from the documents. Include Metadata if available.")



client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  
    ),
    mode=instructor.Mode.JSON,
)

mdoel_embed = "mxbai-embed-large"
model_chat = "llama3.1:8b"
db_path = "markdown_db"
source_dir = r"C:\Users\shive\OneDrive\Documents\DnD\Adventures\Stone-Heart Hollow"
top_k = 5

collection_name = os.path.basename(source_dir)
chroma_rag = ChromaRag(
        source_directory=source_dir,
        collection_name=collection_name,
        db_path=db_path,
        model_name=mdoel_embed
    )

question = "What was in the letter sent to Councillor Bren?"
docs, ids, metadata = chroma_rag.retrieve(question, k=top_k)
docs_text = "\n---\n".join(docs)
# print(docs)
prompt = f"""
    ==================
    You are a virtual assistant for a Dungeon Master running Dungeons & Dragons (D&D) sessions.

    - Helping prepare and run sessions by providing ideas, rules clarifications, encounter design, and narrative suggestions.

    - Answering questions using provided campaign documents, focusing only on the relevant content.

    - If no documents are provided, rely on your general D&D knowledge (primarily 5th Edition unless specified).

    Be accurate, concise, and helpful. Prioritize clarity when referencing documents. Prioritize clarity and creativity when assisting with gameplay and storytelling.
    ==================

    ==================
    DOCUMENT CONTEXT:
    {docs_text}
    ==================

    ==================
    Query:
    {question}
    ==================
"""

resp = client.chat.completions.create(
    model="gemma3",
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    response_model=Answer,
)
print(resp.model_dump_json(indent=2))


{'ids': [['adventure-md-6', 'gallowmere-md-8', 'rumours-md-4', 'rumours-md-7', 'rumours-md-1']], 'embeddings': None, 'documents': [['Title: 🐍 Gallowmere Hollow\nSection: Letter from Kern to Councillor Bren\nSubsection: Untitled\nSource: adventure-md\n\nContent:\n## Letter from Kern to Councillor Bren  \n**To Councilor Bren,**  \nYou sealed the wound, but not the rot beneath.  \nThe stone remembers. The earth watches. You built your town atop sacred ground and buried the truth beneath rock and silence.  \nI will open the path. Judgment is coming.  \nFlee the Hollow. Flee the town—or face a stone-cold gaze and an eternal fate beyond hope.  \n— _Kern Vallis_\n_The Stone Prophet_  \n---', 'Title: Untitled\nSection: 🏙️ Locations in and around Gallowmere\nSubsection: 🪙 Bren’s Office\nSource: gallowmere-md\n\nContent:\n### 🪙 Bren’s Office\nA fortified stone hall serving as both town hall and panic room.\nMaps, letters, and a locked chest lie within. A draft seeps from a wall crack where no se